In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun
import numpy as np


In [3]:
Spark=SparkSession.builder.getOrCreate()

In [4]:
df=Spark.read.csv("/data/titanic.csv",header=True , inferSchema=True)

In [5]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [6]:
numOf_cabin_null=df.filter(fun.col('Cabin').isNull()).count()
numOf_cabin_null

687

In [7]:
avg_ages=df.select( fun.avg("Age")).collect()
avg_ages[0][0]

29.69911764705882

In [8]:
df=df.fillna({'Age':avg_ages[0][0]})

In [9]:
passengers_who_survived = df.filter(df['Survived']== 1).count()
passengers_who_Notsurvived=df.filter(df['Survived']== 0).count()
print(f"total number of passengers who survived: {passengers_who_survived} \nand those who did not: {passengers_who_Notsurvived}")

total number of passengers who survived: 342 
and those who did not: 549


In [10]:
embarkation_ports=df.groupBy('Embarked').count().orderBy(fun.desc("count"))
embarkation_ports.show(5)



+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



In [11]:
df.groupBy('Pclass').agg(fun.avg('Survived').alias("SurvivalRate")).show()

+------+-------------------+
|Pclass|       SurvivalRate|
+------+-------------------+
|     1| 0.6296296296296297|
|     3|0.24236252545824846|
|     2|0.47282608695652173|
+------+-------------------+



In [15]:
df.select(fun.max('Fare')).show()

+---------+
|max(Fare)|
+---------+
| 512.3292|
+---------+



In [16]:
df.select(fun.min('Fare')).show()

+---------+
|min(Fare)|
+---------+
|      0.0|
+---------+



In [17]:
df.select(fun.avg('Fare')).show()

+----------------+
|       avg(Fare)|
+----------------+
|32.2042079685746|
+----------------+



In [26]:
age_0_18 ,age_19_35, age_36_60,age_plus60 =0,0,0,0
age_0_18 = df.filter(df['Age'] <= 18).count()
age_19_35 = df.filter(  (df['Age'] >=19)  & (df['Age'] <= 35)).count()
age_36_60 = df.filter(  (df['Age'] >=36)  & (df['Age'] <= 60)).count()
age_plus60 = df.filter(df['Age'] > 60).count()

print(f"0-18: {age_0_18}, 19-35: {age_19_35}, 36-60: {age_36_60}, 60+: {age_plus60}")



0-18: 139, 19-35: 535, 36-60: 195, 60+: 22


In [32]:
df.write.option("header", True).csv("/data/processed_output")


